In [1]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()


2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT *
FROM 'Facilities'
WHERE membercost <> 0.0;

/* Q2: How many facilities do not charge a fee to members? */

4

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost < ( monthlymaintenance * 0.2 );

/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT * 
FROM  `Facilities` 
WHERE facid
IN ( 1, 5 );

/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT 
    name, 
    monthlymaintenance
    CASE WHEN monthlymaintenance >100
    THEN  'expensive'
    ELSE  'cheap'
    END AS maintenance_cat
FROM  `Facilities`

/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT surname, firstname, joindate
FROM Members
ORDER BY joindate DESC

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

SELECT DISTINCT  
    f.name AS Facility
    CONCAT( m.surname,  ', ', m.firstname) AS mem_name
FROM Members AS m
    INNER JOIN Bookings AS b ON b.memid = m.memid
    INNER JOIN Facilities AS f ON b.facid = f.facid
WHERE b.facid IN ( 0, 1 ) AND b.facid !=0
ORDER BY mem_name

/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT 
    f.name AS Facility, 
    CONCAT(m.surname, ', ', m.firstname) AS Member_Name, 
    CASE WHEN b.memid >0 THEN (b.slots * f.membercost)
    ELSE (b.slots * f.guestcost)
    END AS Total_Cost
FROM Bookings AS b
    INNER JOIN Members AS m ON b.memid = m.memid
    INNER JOIN Facilities AS f ON b.facid = f.facid
WHERE CAST(starttime AS DATE) = CAST('2012-09-14' AS DATE) AND 
    CASE WHEN b.memid >0 THEN (b.slots * f.membercost)
    ELSE (b.slots * f.guestcost)
    END >30
ORDER BY Total_Cost

/* Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT 
    sub.name, 
    CONCAT(sub.firstname, ' ', sub.surname),
    CASE WHEN sub.memid != 0 THEN sub.slots * sub.membercost
    ELSE sub.slots * sub.guestcost
    END AS Total_Cost
FROM (SELECT b.starttime, b.facid, b.memid, b.slots, f.membercost, f.guestcost, f.name,
      m.firstname, m.surname
      FROM Bookings as b
      INNER JOIN Members as m
      ON b.memid = m.memid
      INNER JOIN Facilities as f
      ON f.facid = b.facid) as sub
WHERE CAST(starttime AS date) = CAST('2012-09-14' AS date) AND
    CASE WHEN memid != 0 THEN slots * membercost
    ELSE slots * guestcost
    END > 30
ORDER BY Total_Cost DESC

/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [3]:
Query1 = """SELECT
    name, 
    SUM(rev) as total_revenue 
FROM (SELECT f.name,  
       CASE WHEN b.memid != 0 THEN f.membercost*b.slots 
            ELSE f.guestcost*b.slots END AS rev
FROM Facilities as f 
INNER JOIN Bookings as b
ON b.facid = f.facid) as sub
GROUP BY sub.name
HAVING SUM(rev) < 1000
ORDER BY total_revenue;"""

con = sqlite3.connect("sqlite_db_pythonsqlite.db")
cur = con.cursor()

cur.execute(Query1)

rows = cur.fetchall()
 
for row in rows:
        print(row)


('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)


In [ ]:
/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [4]:
Query2 = """SELECT
    firstname,
    surname,
    (SELECT firstname from Members as m2 where m2.memid = m1.recommendedby and m1.recommendedby != 0) as Reco_first, 
    (SELECT surname from Members as m2 where m2.memid = m1.recommendedby and m1.recommendedby != 0) as Reco_surname
FROM Members as m1
WHERE memid != 0
ORDER BY surname, firstname;"""

con = sqlite3.connect("sqlite_db_pythonsqlite.db")
cur = con.cursor()

cur.execute(Query2)

rows = cur.fetchall()
 
for row in rows:
        print(row)

('Florence', 'Bader', 'Ponder', 'Stibbons')
('Anne', 'Baker', 'Ponder', 'Stibbons')
('Timothy', 'Baker', 'Jemima', 'Farrell')
('Tim', 'Boothe', 'Tim', 'Rownam')
('Gerald', 'Butters', 'Darren', 'Smith')
('Joan', 'Coplin', 'Timothy', 'Baker')
('Erica', 'Crumpet', 'Tracy', 'Smith')
('Nancy', 'Dare', 'Janice', 'Joplette')
('David', 'Farrell', None, None)
('Jemima', 'Farrell', None, None)
('Matthew', 'Genting', 'Gerald', 'Butters')
('John', 'Hunt', 'Millicent', 'Purview')
('David', 'Jones', 'Janice', 'Joplette')
('Douglas', 'Jones', 'David', 'Jones')
('Janice', 'Joplette', 'Darren', 'Smith')
('Anna', 'Mackenzie', 'Darren', 'Smith')
('Charles', 'Owen', 'Darren', 'Smith')
('David', 'Pinker', 'Jemima', 'Farrell')
('Millicent', 'Purview', 'Tracy', 'Smith')
('Tim', 'Rownam', None, None)
('Henrietta', 'Rumney', 'Matthew', 'Genting')
('Ramnaresh', 'Sarwin', 'Florence', 'Bader')
('Darren', 'Smith', None, None)
('Darren', 'Smith', None, None)
('Jack', 'Smith', 'Darren', 'Smith')
('Tracy', 'Smith', N

In [ ]:
/* Q12: Find the facilities with their usage by member, but not guests */

In [6]:
Query3 = """
SELECT 
    f.name, 
    count(*)
FROM Facilities AS f
INNER JOIN Bookings AS b
ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY f.name;"""

con = sqlite3.connect("sqlite_db_pythonsqlite.db")
cur = con.cursor()

cur.execute(Query3)

rows = cur.fetchall()
 
for row in rows:
        print(row)

('Badminton Court', 344)
('Massage Room 1', 421)
('Massage Room 2', 27)
('Pool Table', 783)
('Snooker Table', 421)
('Squash Court', 195)
('Table Tennis', 385)
('Tennis Court 1', 308)
('Tennis Court 2', 276)


In [ ]:
/* Q13: Find the facilities usage by month, but not guests */

In [8]:
Query4 = """
SELECT 
    f.name, 
    (strftime('%Y-%m', starttime)) as month_count,
    count(*)
FROM Facilities AS f
INNER JOIN Bookings AS b
ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY f.name, month_count;"""

con = sqlite3.connect("sqlite_db_pythonsqlite.db")
cur = con.cursor()

cur.execute(Query4)

rows = cur.fetchall()
 
for row in rows:
        print(row)

('Badminton Court', '2012-07', 51)
('Badminton Court', '2012-08', 132)
('Badminton Court', '2012-09', 161)
('Massage Room 1', '2012-07', 77)
('Massage Room 1', '2012-08', 153)
('Massage Room 1', '2012-09', 191)
('Massage Room 2', '2012-07', 4)
('Massage Room 2', '2012-08', 9)
('Massage Room 2', '2012-09', 14)
('Pool Table', '2012-07', 103)
('Pool Table', '2012-08', 272)
('Pool Table', '2012-09', 408)
('Snooker Table', '2012-07', 68)
('Snooker Table', '2012-08', 154)
('Snooker Table', '2012-09', 199)
('Squash Court', '2012-07', 23)
('Squash Court', '2012-08', 85)
('Squash Court', '2012-09', 87)
('Table Tennis', '2012-07', 48)
('Table Tennis', '2012-08', 143)
('Table Tennis', '2012-09', 194)
('Tennis Court 1', '2012-07', 65)
('Tennis Court 1', '2012-08', 111)
('Tennis Court 1', '2012-09', 132)
('Tennis Court 2', '2012-07', 41)
('Tennis Court 2', '2012-08', 109)
('Tennis Court 2', '2012-09', 126)
